---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    result = [] 
    state = None
    with open('university_towns.txt') as file:
        for line in file: 
            line = line[:-1]
            if '[edit]' in line: 
                state = line.split('[')[0]
                continue
            if '(' in line:
                region = line.split(' (')[0]
                result.append([state, region])
            else: 
                region = line
                result.append([state, region])

    df = pd.DataFrame(result, columns = ["State", "RegionName"])

    return df


In [4]:
def get_gdp_df():
    '''
    Create a pandas dataframe containing the Quarter and GDP columns 
    for the years 2000 or later. 
    '''

    gdp = pd.ExcelFile("gdplev.xls")
    gdp = gdp.parse("Sheet1", skiprows=219)
    gdp = gdp[["1999q4", 9926.1]]
    gdp.columns = ["Quarter", "GDP"]

    return gdp


def get_recession_indices():
    '''
    With the GDP dataframe, get a list of the indices of the DataFrame
    where there is recession 
    '''

    gdp = get_gdp_df()
    index_list = [] 
    for i in range(len(gdp)-2):
        if (gdp.iloc[i][1] > gdp.iloc[i+1][1]) and (gdp.iloc[i+1][1] > gdp.iloc[i+2][1]):
            if (i+2) not in index_list:
                index_list.append(i)
                index_list.append(i+1)
                index_list.append(i+2)
                
    return index_list


def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''

    indices = get_recession_indices()
    gdp = get_gdp_df()
    start_index = indices[0]
    result = gdp.iloc[start_index][0]
    
    return result



In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    indices = get_recession_indices()   
    last = indices[-1]
    result = gdp.iloc[last+1][0]

    return result
    


In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    indices = get_recession_indices()
    gdp = get_gdp_df()
    end = indices[-1]
    result = gdp.iloc[end][0]

    return result

In [7]:
def year_qtr_columns():
    
    years = list(range(2000,2017))
    quarters = ["q" + str(num) for num in list(range(1,5))]
    cols = []

    for year in years: 
        for q in quarters: 
            cols.append(str(year)+q)
    del cols[-1]

    return cols 


def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    df.drop(['Metro','CountyName','RegionID','SizeRank'],axis=1,inplace=1)
    df['State'] = df['State'].map(states)
    df.set_index(["State", "RegionName"], inplace=True)
    drop_cols = [c for c in df.columns if c.lower()[:3] == '199']
    df.drop(drop_cols, axis=1, inplace=1) 
    qtrs = [list(df.columns)[x:x+3] for x in range(0, len(list(df.columns)), 3)]
    cols = year_qtr_columns()

    # getting the mean of the months to get quarterly instead of monthly data
    for col, q in zip(cols, qtrs):
        df[col] = df[q].mean(axis=1)
        
    df = df[cols]

    return df

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence.     

    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''

    
    start = get_recession_start()
    bottom = get_recession_bottom()

    df = convert_housing_data_to_quarters().reset_index()
    df = df[["RegionName", start, bottom]]
    
    # This is the correct formula for price ratio, not the one listed
    # at the top of the assignment. 
    
    df["PriceRatio"] = (df[start] - df[bottom])/df[start]

    uni_towns = get_list_of_university_towns()["RegionName"]    

    uni = df.loc[df["RegionName"].isin(uni_towns)]
    uni = uni.set_index("RegionName")
    uni = uni["PriceRatio"].dropna()
    
    not_uni = df.loc[~df["RegionName"].isin(uni_towns)]
    not_uni = not_uni.set_index("RegionName")
    not_uni = not_uni["PriceRatio"].dropna()

    p = ttest_ind(uni, not_uni)[1]

    if p<0.01:
        diff = True
    else: 
        diff = False

    better = "university town"
    if uni.mean() > not_uni.mean():
        better = "non-university town"

    return (diff,p, better)

run_ttest()